<a href="https://colab.research.google.com/github/anruki/AthenAI_Competition/blob/main/Data_Inspection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inspection of Input data for the competition

This Notebook serves as first step of the challenge:
* Data inspection

The purpose of the following code is to examine the Input data and explain upcoming challenges for the competition.

## Preparing the Environment

1. Import Libraries

In [ ]:
import pandas as pd
import zipfile
import os

2. Extract zip file

In [4]:
zip_file_path = '/content/datos_competicion.zip'
extraction_path = '/content/datos'

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print(f"'{zip_file_path}' unzipped to '{extraction_path}'")

'/content/datos_competicion.zip' unzipped to '/content/datos'


## Algoritmos Folder Inspection

The 'algoritmos' folder contains multiple csv

In [3]:
extraction_path = '/content/datos'
algoritmos_path = os.path.join(extraction_path, 'algoritmos')

# Get a list of all files in the 'algoritmos' folder
algo_files = [f for f in os.listdir(algoritmos_path) if f.endswith('.csv')]
algo_files.sort() # Sort to ensure consistent 'first' file if not already sorted

if algo_files:
    first_algo_csv = os.path.join(algoritmos_path, algo_files[0])
    print(f"Displaying the first CSV from '{algoritmos_path}': {algo_files[0]}")
    df_first_algo = pd.read_csv(first_algo_csv)
    display(df_first_algo.head())
else:
    print(f"No CSV files found in '{algoritmos_path}'")


Displaying the first CSV from '/content/unzipped_data/algoritmos': 007XY.csv


,datetime,open,high,low,close
0,2022-03-14 21:00:00,100.61,100.61,100.61,100.61
1,2022-03-15 21:00:00,100.61,100.61,100.61,100.61
2,2022-03-16 21:00:00,100.61,100.61,100.61,100.61
3,2022-03-17 21:00:00,100.61,100.61,100.61,100.61
4,2022-03-18 17:00:00,100.61,100.61,100.61,100.61


In [ ]:
if 'algo_files' in globals():
    num_algo_csv = len(algo_files)
    print(f"There are {num_algo_csv} CSV files in the 'algoritmos' folder.")
else:
    print("The 'algo_files' variable was not found. Please ensure the previous cell for inspecting the 'algoritmos' folder has been executed.")


In [ ]:
import pandas as pd
import os

extraction_path = '/content/unzipped_data'

# Define paths for the three main CSV files
benchmark_monthly_path = os.path.join(extraction_path, 'benchmark_monthly_returns.csv')
benchmark_yearly_path = os.path.join(extraction_path, 'benchmark_yearly_returns.csv')
trades_benchmark_path = os.path.join(extraction_path, 'trades_benchmark.csv')

print("\nDisplaying 'benchmark_monthly_returns.csv':")
df_benchmark_monthly = pd.read_csv(benchmark_monthly_path)
display(df_benchmark_monthly.head())

print("\nDisplaying 'benchmark_yearly_returns.csv':")
df_benchmark_yearly = pd.read_csv(benchmark_yearly_path)
display(df_benchmark_yearly.head())

print("\nDisplaying 'trades_benchmark.csv':")
df_trades_benchmark = pd.read_csv(trades_benchmark_path)
display(df_trades_benchmark.head())
